In [2]:
import pandas as pd
import pickle

In [3]:
file_path = '../Data/RAW_activePlayerCareerLogData.pickle'
PLAYER = 'kyrie irving'

In [4]:
# Load collected player data from pickle file
with open(file_path, 'rb') as file:
    activePlayerCareerLog = pickle.load(file)

In [41]:
# Clean Data and add basic features

for name, df_player in activePlayerCareerLog.items():
    df_player['WL'].replace({'L': 0, 'W': 1}, inplace=True)
    df_player['REST'] = 0
    
    for idx in df_player.index[1:]:
        date_difference = df_player['GAME_DATE'].iloc[idx] - df_player['GAME_DATE'].iloc[idx - 1]
        df_player.at[idx, 'REST'] = date_difference.days

    df_player.at[0, 'REST'] = 4 # assumed rest from NBA preseason to season start

### Time Series Feature Engineering

In [5]:
activePlayerCareerLog['aaron gordon'].head()

,SEASON_ID,Player_ID,GAME_DATE,WL,MIN,FGM,FGA,FG3M,FG3A,FTM,...,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,HOME,OPP,TEAM
0,22016,203932,2016-10-26,L,35,5,12,2,3,0,...,4,1,1,3,1,12,-11,1,MIA,ORL
1,22016,203932,2016-10-28,L,29,6,12,1,2,4,...,0,2,0,1,3,17,-10,0,DET,ORL
2,22016,203932,2016-10-29,L,26,3,7,1,3,1,...,2,3,1,2,3,8,-21,0,CLE,ORL
3,22016,203932,2016-11-01,W,32,3,10,0,4,1,...,4,3,0,2,1,7,11,0,PHI,ORL
4,22016,203932,2016-11-03,W,31,5,10,1,3,2,...,3,1,1,1,3,13,2,1,SAC,ORL


In [6]:
target = 'PTS'
cols_to_drop = ['SEASON_ID', 'Player_ID', 'GAME_DATE', 'OPP', 'TEAM', target + '_target']

In [10]:
# one lag time step
def shift_target(df, target):
    df[target + '_target'] = df[target].shift(-1)
    df.dropna(inplace=True)
    return df

In [11]:
# rolling window
def recent_average(df, target, games=5):
    shifted_stat = df[target]
    window = shifted_stat.rolling(window=games).mean()
    df['LAST_' + str(games) + '_' + target] = window
    return df

def recent_percentage(df, target1, target2, games=5):
    shifted_stat1 = df[target1]
    shifted_stat2 = df[target2]
    stat_percentage = shifted_stat1 / shifted_stat2
    window = stat_percentage.rolling(window=games).mean()
    df['LAST_' + str(games) + '_' + target1 + '_PCT'] = window
    return df

In [14]:
df_player = activePlayerCareerLog[PLAYER].copy()
df_player = shift_target(df_player, target)
df_player = recent_average(df_player, target, 3)
df_player = recent_average(df_player, target, 5)
df_player = recent_average(df_player, target, 7)
df_player = recent_average(df_player, 'PLUS_MINUS', 3)
df_player = recent_percentage(df_player, 'FGM', 'FGA', 3)

In [17]:
df_player.iloc[-1]

SEASON_ID                          22023
Player_ID                         202681
GAME_DATE            2024-02-12 00:00:00
WL                                     W
MIN                                   43
FGM                                   10
FGA                                   19
FG3M                                   2
FG3A                                   7
FTM                                    4
FTA                                    6
OREB                                   1
DREB                                   6
REB                                    7
AST                                    4
STL                                    0
BLK                                    2
TOV                                    2
PF                                     3
PTS                                   26
PLUS_MINUS                             3
HOME                                   1
OPP                                  WAS
TEAM                                 DAL
PTS_target      

In [48]:
for player in activePlayerCareerLog.values():
    player = shift_target(player)
    player = recent_average(player, target, 3)
    player = recent_average(player, target, 5)
    player = recent_average(player, target, 7)
    player = recent_average(player, 'PLUS_MINUS', 3)
    player = recent_percentage(player, 'FGM', 'FGA', 5)

precious achiuwa
steven adams
bam adebayo
ochai agbaji
santi aldama
nickeil alexander-walker
grayson allen
jarrett allen
jose alvarado
kyle anderson
giannis antetokounmpo
thanasis antetokounmpo
cole anthony
og anunoby
ryan arcidiacono
deni avdija
deandre ayton
udoka azubuike
marvin bagley iii
amari bailey
patrick baldwin jr.
lamelo ball
lonzo ball
mo bamba
paolo banchero
desmond bane
dalano banton
dominick barlow
harrison barnes
scottie barnes
rj barrett
charles bassey
emoni bates
keita bates-diop
nicolas batum
bradley beal
malik beasley
marjon beauchamp
davis bertans
patrick beverley
saddiq bey
goga bitadze
bismack biyombo
bogdan bogdanovic
bojan bogdanovic
bol bol
marques bolden
devin booker
brandon boston jr.
chris boucher
james bouknight
malaki branham
christian braun
mikal bridges
miles bridges
oshae brissett
malcolm brogdon
armoni brooks
dillon brooks
bruce brown
jaylen brown
kendall brown
moses brown
greg brown iii
charlie brown jr.
troy brown jr.
jalen brunson
thomas bryant
reg

In [50]:
# Save collected player data into a pickle file
file_path = '../Data/activePlayerCareerLogData.pickle'
with open(file_path, 'wb') as f:
    pickle.dump(activePlayerCareerLog, f)